<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hr_whisper_medium_test_(edit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-e2b5woug
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-e2b5woug
  Resolved https://github.com/huggingface/transformers to commit ebfdb9ca62205279d5019ef1403877461b3b2da4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8269538 sha256=2bea1185f1b36e43f0b14ac56bcd44e15de959a0d09e9178317e0b89cd09e521
  Stored in directory: /tmp/pip-ephem-wheel-cache-34b7d4qa/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hr", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 666
})

In [7]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [21]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:18<00:00, 84.0MiB/s]


In [23]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [24]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [25]:
text

,0
0,na primjer omogućila je mnogim talentiranim lj...
1,potovani predsjedavajui ribarstvo je vana gosp...
2,nastavlja na temelju ovog europskog projekta k...
3,gospoo predsjedavajua dananje izvjee o europsk...
4,bitno je doprinijeti većem povjerenju potrošač...
...,...
661,radi se o čestim blokadama vlasti napuštanju s...
662,napad islamistikih militanata u sjevernoj nige...
663,ne smijemo ostati slijepi na činjenicu da i eu...
664,u mnogim zemljama ako niste uhvaćeni u počinje...


In [26]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [27]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,female,na primjer omogućila je mnogim talentiranim lj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00286865234375, 0.003021240234375, 0.002655...",16000
1,male,potovani predsjedavajui ribarstvo je vana gosp...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, 0.000152587890625, -0.0006408...",16000
2,male,nastavlja na temelju ovog europskog projekta k...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, -0.00018310546875, 9.1552734...",16000
3,male,gospoo predsjedavajua dananje izvjee o europsk...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00360107421875, 0.002685546875, -0.00036621...",16000
4,female,bitno je doprinijeti većem povjerenju potrošač...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.001190185546875, 0.00189...",16000
...,...,...,...,...,...
661,male,radi se o čestim blokadama vlasti napuštanju s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0023193359375, -0.0010986328125, -0.0001831...",16000
662,male,napad islamistikih militanata u sjevernoj nige...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.039825439453125, 0.027557373046875, 0.02493...",16000
663,female,ne smijemo ostati slijepi na činjenicu da i eu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007049560546875, 0.001708984375, -0.0053100...",16000
664,male,u mnogim zemljama ako niste uhvaćeni u počinje...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.000274658203125, 0.00036...",16000


In [28]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
1,male,potovani predsjedavajui ribarstvo je vana gosp...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, 0.000152587890625, -0.0006408...",16000
2,male,nastavlja na temelju ovog europskog projekta k...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, -0.00018310546875, 9.1552734...",16000
3,male,gospoo predsjedavajua dananje izvjee o europsk...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00360107421875, 0.002685546875, -0.00036621...",16000
5,male,takva je pozicija neizbjena ne samo zbog injen...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.007781982421875, -0.009033203125, -0.00741...",16000
6,male,uloga europe mora i dalje biti promicanje ljud...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.004669189453125, -0.00543212890625, -0.003...",16000
...,...,...,...,...,...
658,male,stoga sam uvjeren da upravo visoki standardi z...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.009552001953125, -0.018829345703125, -0.01...",16000
659,male,od trenutka pristupanja hrvatske u punopravno ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00042724609375, 0.005462646484375, 0.00509...",16000
661,male,radi se o čestim blokadama vlasti napuštanju s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0023193359375, -0.0010986328125, -0.0001831...",16000
662,male,napad islamistikih militanata u sjevernoj nige...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.039825439453125, 0.027557373046875, 0.02493...",16000


In [29]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
0,female,na primjer omogućila je mnogim talentiranim lj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00286865234375, 0.003021240234375, 0.002655...",16000
4,female,bitno je doprinijeti većem povjerenju potrošač...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.001190185546875, 0.00189...",16000
8,female,dakle sankcije treba uzeti kao mjere koje nisu...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0001220703125, -0.000762939453125, -0.0001...",16000
9,female,ako se ovako nastavi stradat će proizvodnja hr...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000396728515625, 0.000152587890625, 0.0006...",16000
11,female,za ostvarivanje ambicioznih ciljeva potrebno j...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001434326171875, 0.0003662109375, -0.000732...",16000
...,...,...,...,...,...
653,female,kao zastupnica iz hrvatske a svjesna alarmantn...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, 0.000640869140625, 0.00030...",16000
657,female,niti jedna drava lanica europske unije nije do...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002655029296875, -0.0084228515625, -0.0046...",16000
660,female,kolegice vi jako dobro znate da neke države čl...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00048828125, -0.0006103515625, -0.00018310...",16000
663,female,ne smijemo ostati slijepi na činjenicu da i eu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007049560546875, 0.001708984375, -0.0053100...",16000


In [30]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hr", fp16=False)
    return result["text"]

In [31]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [32]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [33]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 2 files, WER: 0.3793103448275862, Precision: 0.5833333333333334, Recall: 0.5833333333333334, F1: 0.5833333333333334
Processed 3 files, WER: 0.5142857142857142, Precision: 0.46875, Recall: 0.5172413793103449, F1: 0.4918032786885246
Processed 4 files, WER: 0.5434782608695652, Precision: 0.46153846153846156, Recall: 0.43902439024390244, F1: 0.45
Processed 5 files, WER: 0.3684210526315789, Precision: 0.6842105263157895, Recall: 0.7222222222222222, F1: 0.7027027027027027
Processed 6 files, WER: 0.575, Precision: 0.5161290322580645, Recall: 0.42105263157894735, F1: 0.46376811594202894
Processed 7 files, WER: 0.6, Precision: 0.6666666666666666, Recall: 0.4, F1: 0.5
Processed 8 files, WER: 0.20689655172413793, Precision: 0.782608695652174, Recall: 0.782608695652174, F1: 0.782608695652174
Processed 9 files, WER: 0.23076923076923078, Precision: 0.7647058823529411, Recall: 0.7647058823529411, F1: 0.7647058823529412
Processe

In [34]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)


    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.14285714285714285, Precision: 0.8571428571428571, Recall: 0.84, F1: 0.8484848484848485
Processed 2 files, WER: 0.3888888888888889, Precision: 0.6111111111111112, Recall: 0.6111111111111112, F1: 0.6111111111111112
Processed 3 files, WER: 0.3333333333333333, Precision: 0.6451612903225806, Recall: 0.6896551724137931, F1: 0.6666666666666667
Processed 4 files, WER: 0.26666666666666666, Precision: 0.7142857142857143, Recall: 0.7142857142857143, F1: 0.7142857142857143
Processed 5 files, WER: 0.3, Precision: 0.7, Recall: 0.7, F1: 0.7
Processed 6 files, WER: 0.4166666666666667, Precision: 0.7777777777777778, Recall: 0.6363636363636364, F1: 0.7000000000000001
Processed 7 files, WER: 0.25, Precision: 0.7368421052631579, Recall: 0.7368421052631579, F1: 0.7368421052631579
Processed 8 files, WER: 0.3333333333333333, Precision: 0.7272727272727273, Recall: 0.7272727272727273, F1: 0.7272727272727273
Processed 9 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 10 f